In [14]:
%run ANNOTATIONS.ipynb

In [35]:
%run PATTERN_OBSERVER.ipynb

In [22]:
# ability to make backtest checkbox and book checkbox lists
%run CHECKBOXES.ipynb

In [24]:
# location of datafiles
%run DATA_CONTAINER.ipynb

In [18]:
# ability to make settings block
%run SETTINGS.ipynb

SETTINGS : REFERENCE : all_backtests : {'AZUL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/AZUL4.SA.csv'), 'BEEF3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BEEF3.SA.csv'), 'BRFS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'), 'CCRO3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/CCRO3.SA.csv'), 'ECOR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/ECOR3.SA.csv'), 'EMBR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/EMBR3.SA.csv'), 'GOLL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/GOLL4.SA.csv'), 'JBSS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/JBSS3.SA.csv'), 'MRFG3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/MRFG3.SA.csv'), 'RAIL3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/RAIL3.SA.csv'), 'VALE3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'), 'WEGE3.SA': WindowsPath('C:/Users/ahkar/OneDriv

In [7]:
# define extraday analysis plots
%run VIEWING_TABS_EXTRADAY_ANALYSIS.ipynb

viewing tab = extraday evolution


In [8]:
# define intraday analysis plots
%run VIEWING_TABS_INTRADAY_ANALYSIS.ipynb

viewing tab = intraday evolution


In [9]:
# define factor analysis plots
%run VIEWING_TABS_FACTOR_ANALYSIS.ipynb

viewing tab = factor analysis


In [25]:
class BACKTEST_VIEWER(IObserver):
    def __init__(self,
        name='BACKTEST_VIEWER'
        ):
        self.name=name
        
        # instantiate SETTINGS
        self._settings=SETTINGS(
            name='SETTINGS',
            width='20%',
        )
        self._view_type=self.applied_view_type

        # instantiate BACKTEST_SELECTOR
        self._backtests=CHECKBOXES(
            name='BACKTEST_SELECTOR',
            options=self._settings.reference['all_backtests'],
            default=False,
            width='60%',
        )

        # instantiate BOOK_SELECTOR
        self._books=CHECKBOXES(
            name='BOOK_SELECTOR',
            options=self._settings.reference['all_books'],                                                           # TODO `BACKTEST SELECTOR` needs to have `all_books`
            default=True,
            width='20%',
        )
        #self._backtests.attach(self._books)                                                                         # TODO `BOOK SELECTOR` needs to observe changes in `BACKTEST SELECTOR|all_backtests`

        # instantiate DATA_CONTAINER
        self._data_container=DATA_CONTAINER(                                                                                                # TODO needs to observe changes in `self.selected_backtests`, `self.selected_books` and `self.applied_settings`
            applied_backtest_path=self._applied_backtest_path,
            applied_view_type=self._applied_view_type,
            applied_return_type=self._applied_return_type,
            applied_date_from=self._applied_date_from,
            applied_date_to=self._applied_date_to,
            selected_backtests=self._selected_backtests,
            selected_books=self._selected_books,
        )
        
        # BACKTEST_SELECTOR observes ...
        self._settings.attach(self._backtests)       # `all_backtests` from SETTINGS

        # DATA_CONTAINER observes ...
        self._settings.attach(self._data_container)  # `backtest_path` from SETTINGS
        self._backtests.attach(self._data_container) # `applied_settings` from BACKTEST_SELECTOR, i.e. changes in selected backtests
        self._books.attach(self._data_container)     # `applied_settings` from BOOK_SELECTOR, i.e. changes in selected books

        # build
        self.widget=self._make()                                                                                        # TODO needs to observe changes in self._data_container

    ####################################
    # observer pattern
    ####################################
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info)

        # SETTINGS changed
        if subject_name=='SETTINGS':
            if self._view_type!=self.subject_info['view_type']:

                # VIEWING TABS observe DATA_CONTAINER
                self._data_container.attach(EXTRADAY_TABS)
                self._data_container.attach(INTRADAY_TABS)
                self._data_container.attach(FACTOR_TABS)

    @property
    def _applied_backtest_path(self) -> str :
        return self._settings.reference['backtest_path']

    @property
    def _applied_view_type(self) -> str :
        return self._settings.reference['view_type']

    @property
    def _applied_return_type(self) -> str :
        return self._settings.reference['return_type']

    @property
    def _applied_date_from(self) -> object :
        return self._settings.reference['date_from']

    @property
    def _applied_date_to(self) -> object :
        return self._settings.reference['date_to']

    @property
    def _selected_backtests(self) -> OPTIONID_VALUES:
        return self._backtests.applied_settings
        
    @property
    def _selected_books(self) -> OPTIONID_VALUES:
        return self._books.applied_settings

    @property
    def _control_widgets(self) -> BUTTONS:
        return self.widget.children[0].children
    
    @property
    def _viewing_widgets(self) -> WIDGETS:
        return self.widget.children[1].children

    ####################################
    # GUI
    ####################################
    def _make(self) -> VBox :
        # controls
        controls=HBox(
            [
                self._backtests.widget,
                self._books.widget,
                self._settings.widget,
            ]
        )
        
        # viewing gab
        self._make_viewing_tab()

        # build screen
        return VBox(
            [
                HBox(controls),
                self._viewing_block.widget
            ]
        )
    
    def _make_viewing_tab(self) -> TAB_COLLECTION :
        # instantiate
        self._viewing_tab=make_extraday_tab()
        
        # VIEWING_TAB observes ...
        self._settings.attach(self._viewing_tab)                  # `view_type` from SETTINGS

        # register checkboxes
        for c in widgets:
            c.observe(self._onchange_checkbox,names='value')

# example
BACKTEST_VIEWER().widget

SETTINGS : REFERENCE : all_backtests : {'AZUL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/AZUL4.SA.csv'), 'BEEF3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BEEF3.SA.csv'), 'BRFS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'), 'CCRO3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/CCRO3.SA.csv'), 'ECOR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/ECOR3.SA.csv'), 'EMBR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/EMBR3.SA.csv'), 'GOLL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/GOLL4.SA.csv'), 'JBSS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/JBSS3.SA.csv'), 'MRFG3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/MRFG3.SA.csv'), 'RAIL3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/RAIL3.SA.csv'), 'VALE3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'), 'WEGE3.SA': WindowsPath('C:/Users/ahkar/OneDriv

In [39]:
'a' | None

TypeError: unsupported operand type(s) for |: 'str' and 'NoneType'

In [27]:
# display
a=BACKTEST_VIEWER()
a.widget

SETTINGS : REFERENCE : all_backtests : {'AZUL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/AZUL4.SA.csv'), 'BEEF3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BEEF3.SA.csv'), 'BRFS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/BRFS3.SA.csv'), 'CCRO3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/CCRO3.SA.csv'), 'ECOR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/ECOR3.SA.csv'), 'EMBR3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/EMBR3.SA.csv'), 'GOLL4.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/GOLL4.SA.csv'), 'JBSS3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/JBSS3.SA.csv'), 'MRFG3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/MRFG3.SA.csv'), 'RAIL3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/RAIL3.SA.csv'), 'VALE3.SA': WindowsPath('C:/Users/ahkar/OneDrive/Documents/Data/B3/VALE3.SA.csv'), 'WEGE3.SA': WindowsPath('C:/Users/ahkar/OneDriv

In [30]:
a._data_container.reference['data_dico'].keys()

dict_keys(['AZUL4.SA', 'BEEF3.SA', 'BRFS3.SA', 'CCRO3.SA', 'ECOR3.SA', 'EMBR3.SA', 'GOLL4.SA', 'JBSS3.SA', 'MRFG3.SA', 'RAIL3.SA', 'VALE3.SA', 'WEGE3.SA'])

In [32]:
dat=a._data_container.reference['data_dico']
dat

{'AZUL4.SA':      timestamp   open   high    low  close  adjusted_close   volume  \
 0   2021-07-05  44.71  44.99  44.25  44.35           44.35  1602600   
 1   2021-07-02  44.19  44.85  43.54  44.72           44.72  4515900   
 2   2021-07-01  44.02  44.32  43.15  43.96           43.96  4256400   
 3   2021-06-30  44.29  44.69  43.62  43.89           43.89  4981300   
 4   2021-06-29  45.11  45.55  43.70  44.53           44.53  4855400   
 ..         ...    ...    ...    ...    ...             ...      ...   
 95  2021-02-17  42.86  43.60  42.46  43.33           43.33  2863100   
 96  2021-02-12  42.68  43.18  41.80  42.86           42.86  4010100   
 97  2021-02-11  42.91  44.34  42.64  43.04           43.04  5504700   
 98  2021-02-10  44.47  44.65  42.15  42.75           42.75  4654400   
 99  2021-02-09  44.30  44.78  43.71  44.30           44.30  5548600   
 
     dividend_amount  split_coefficient  
 0               0.0                1.0  
 1               0.0                1.

In [34]:
import pickle
a_file = open("data.pkl", "wb")
pickle.dump(dat,a_file)
'''
import pickle
a_file = open("data.pkl", "rb")
output = pickle. load(a_file)
'''

'\na_file = open("data.pkl", "rb")\noutput = pickle. load(a_file)\n'

# notes

settings -> apply

	if date from OR date to changed
		update plots with new date filters

book selector -> apply

	update all plots

backtest selector -> apply

	update all plots

1	dictionary of raw data that has been read
2	filtered data to draw given date from and to (or just change the x-axes)